In [10]:
import numpy as np
import pandas as pd



In [11]:
#this ysed to read the full flights data and extract only the given columns
flights=pd.read_csv("flightsCondensed.csv",usecols=["AIRLINE", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT", "ARRIVAL_DELAY"]) 

len(flights)

C:\Users\smala\AppData\Local\Temp\ipykernel_28904\2520876213.py:2: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  flights=pd.read_csv("flightsCondensed.csv",usecols=["AIRLINE", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT", "ARRIVAL_DELAY"])


5819079

In [12]:
print(flights["ARRIVAL_DELAY"][flights["ORIGIN_AIRPORT"] == "BOS"].mean())

4.29164917369566


In [13]:
flights.to_csv("flightsCondensed.csv", index=False)

In [19]:
airports=pd.read_csv("airports.csv")


In [34]:

# Count the number of flights that either originate from or arrive at the given airport
def countTotalFlights(iata): 
    return np.sum((flights["ORIGIN_AIRPORT"] == iata) | (flights["DESTINATION_AIRPORT"] == iata))

def countFlights(iata, airline):
    if airline == "Other": 
        return countTotalFlights(iata) - countFlights(iata, "UA") - countFlights(iata, "DL") - countFlights(iata, "AA")
    else:
        return np.sum(
            ((flights["ORIGIN_AIRPORT"] == iata) | (flights["DESTINATION_AIRPORT"] == iata))
            & (flights["AIRLINE"] == airline))
    
def avgDelay(iata, airline, inout):
    airportCol = ""
    if inout == "in":
        airportCol = "DESTINATION_AIRPORT"
    if inout == "out":
        airportCol = "ORIGIN_AIRPORT"
    
    if airline == "Other":
        return flights["ARRIVAL_DELAY"][(flights[airportCol] == iata) 
                                        & (flights["AIRLINE"] != "UA")
                                        & (flights["AIRLINE"] != "DL")
                                        & (flights["AIRLINE"] != "AA")].mean()
    else:
        return flights["ARRIVAL_DELAY"][(flights[airportCol] == iata) 
                                        & (flights["AIRLINE"] == airline)].mean()

In [ ]:
airports["TOTAL_FLIGHTS"] = airports.apply (lambda airport : countTotalFlights(airport["IATA_CODE"]), axis=1)
print("total flights done")


In [35]:
airports["UNITED_FLIGHTS"] = airports.apply (lambda airport : countFlights(airport["IATA_CODE"], "UA"), axis=1)
print("united flights done")
print(np.sum(airports["UNITED_FLIGHTS"]))
airports["DELTA_FLIGHTS"] = airports.apply (lambda airport : countFlights(airport["IATA_CODE"], "DL"), axis=1)
print("delta flights done")
airports["AMERICAN_FLIGHTS"] = airports.apply (lambda airport : countFlights(airport["IATA_CODE"], "AA"), axis=1)
print("american flights done")
airports["OTHER_FLIGHTS"] = airports.apply (lambda airport : countFlights(airport["IATA_CODE"], "Other"), axis=1)
print("other flights done")

united flights done
939658
delta flights done
american flights done
other flights done


In [ ]:
airports["UNITED_IN_DELAY"] = airports.apply (lambda airport : avgDelay(airport["IATA_CODE"], "UA", "in"), axis=1)
print("united in-delays done")
airports["DELTA_IN_DELAY"] = airports.apply (lambda airport : avgDelay(airport["IATA_CODE"], "DL", "in"), axis=1)
print("delta in-delays done")
airports["AMERICAN_IN_DELAY"] = airports.apply (lambda airport : avgDelay(airport["IATA_CODE"], "AA", "in"), axis=1)
print("american in-delays done")
airports["OTHER_IN_DELAY"] = airports.apply (lambda airport : avgDelay(airport["IATA_CODE"], "Other", "in"), axis=1)
print("other in-delays done")

airports["UNITED_OUT_DELAY"] = airports.apply (lambda airport : avgDelay(airport["IATA_CODE"], "UA", "out"), axis=1)
print("united out-delays done")
airports["DELTA_OUT_DELAY"] = airports.apply (lambda airport : avgDelay(airport["IATA_CODE"], "DL", "out"), axis=1)
print("delta out-delays done")
airports["AMERICAN_OUT_DELAY"] = airports.apply (lambda airport : avgDelay(airport["IATA_CODE"], "AA", "out"), axis=1)
print("american out-delays done")
airports["OTHER_OUT_DELAY"] = airports.apply (lambda airport : avgDelay(airport["IATA_CODE"], "Other", "out"), axis=1)
print("other out-delays done")

In [37]:
airports.fillna(0, inplace=True)

In [38]:
airports.to_csv("airports.csv", index=False)